In [1]:
import pandas as pd
from Acesso import Login
from Query import Query
from glob import glob

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

In [2]:
arquivos=glob('*.xlsx')

df=pd.read_excel(arquivos[-1])

In [3]:
df

,SKU,Produto,Qtde Caixa,B,C,D,E,F,FUNC,B2C,...,DISTRIBUIDORES,Dif.1,TABELA FUNCIONÁRIO,Dif.2,TABELA CONSUMIDOR FINAL,Dif.3,Desc C,Desc D,Desc E,Desc F
0,3.0,POLPINHA DE ABACAXI (100G),6.0,22.21,20.85,19.64,18.82,18.56,14.89,26.57,...,19.64,0.00,14.89,0.0,26.57,0.0,0.0612,0.1157,0.1526,0.1643
1,12.0,POLPINHA DE ACEROLA (100G),6.0,24.64,23.12,21.78,20.88,20.59,16.52,29.47,...,21.78,0.00,16.52,0.0,29.47,0.0,0.0617,0.1161,0.1526,0.1644
2,35.0,POLPINHA DE AMORA (100G),6.0,52.20,48.98,46.14,44.22,43.62,34.99,62.43,...,46.14,0.00,34.99,0.0,62.43,0.0,0.0617,0.1161,0.1529,0.1644
3,55.0,POLPINHA DE CAJA (100G),6.0,27.33,25.65,24.16,23.15,22.84,18.32,32.69,...,24.16,0.00,18.32,0.0,32.69,0.0,0.0615,0.1160,0.1529,0.1643
4,57.0,POLPINHA DE CAJU (100G),6.0,23.43,21.98,20.71,19.85,19.58,15.70,28.02,...,20.71,0.00,15.70,0.0,28.02,0.0,0.0619,0.1161,0.1528,0.1643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,9628.0,XAROPE DE GUARANA DEMARCHI 5L,1.0,50.49,48.94,47.48,45.44,44.80,34.96,60.02,...,47.48,0.00,34.96,0.0,60.02,0.0,0.0307,0.0596,0.1000,0.1127
419,9629.0,"CONGELADO BATATA MCCAIN RUSTICA IMP (2,5KG)",4.0,58.17,55.68,54.89,54.13,53.39,42.83,68.38,...,54.89,0.00,42.83,0.0,68.38,0.0,0.0428,0.0564,0.0695,0.0822
420,9630.0,CONGELADO POLENTA PALITO DM (1KG),10.0,13.33,12.17,11.83,11.67,11.51,9.23,15.85,...,11.67,-0.16,9.23,0.0,15.85,0.0,0.0870,0.1125,0.1245,0.1365
421,9631.0,CONGELADO POLENTA PALITO DM (300G),20.0,4.60,4.20,4.08,4.03,3.97,3.19,5.47,...,4.03,-0.05,3.19,0.0,5.47,0.0,0.0870,0.1130,0.1239,0.1370


In [20]:
#00000029,00000045,00000004,00000050

#alterar desconto

codigos=df['SKU'].unique().tolist()

conecta=sql.ConexaoSQL()

for c in codigos:
    
    try:
    
        desc=df.loc[df['SKU']==c,'Desc F'].max()

        querys=f"""

        UPDATE preco
        SET desc_prom={float(desc)},
        desc_max_prd={float(desc)}
        WHERE cd_prod={c} AND LTRIM(RTRIM(cd_tabela))='00000029'

        """

        sql.Salvar(querys,conecta)
        
        pass
    
    except Exception as erro:
        
        print(erro)
        
        pass
    
    #break
    
    pass


Erro: ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'nan'. (207) (SQLExecDirectW)")


In [21]:
#alteração de preço massivo
tabelas_dict={'00000029':'B','00000045':'D','00000004':'FUNC','00000050':'B2C'}

codigos=df['SKU'].unique().tolist()

conecta=sql.ConexaoSQL()

for cod,tab in tabelas_dict.items():
    
    for c in codigos:
        
        preco=df.loc[df['SKU']==c,tab].max()

        querys=f"""

        UPDATE preco
        SET vl_preco={float(preco)}
        WHERE cd_prod={c} AND LTRIM(RTRIM(cd_tabela))='{cod}'

        """

        sql.Salvar(querys,conecta)

        #break

        pass
    
    print(f'Tabela {tab} alterada com sucesso!')
    
    pass


Erro: ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'nan'. (207) (SQLExecDirectW)")
Tabela B alterada com sucesso!
Erro: ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'nan'. (207) (SQLExecDirectW)")
Tabela D alterada com sucesso!
Erro: ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'nan'. (207) (SQLExecDirectW)")
Tabela FUNC alterada com sucesso!
Erro: ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'nan'. (207) (SQLExecDirectW)")
Tabela B2C alterada com sucesso!
